In [69]:
import pandas as pd
import numpy as np

data = pd.read_csv(
    'CO2_passenger_cars_v12.csv',
    encoding='utf-8',
    sep='\t'
).reset_index(drop=True)

/home/gustavo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
bymanufacturer = data.groupby('Mh')

df = pd.DataFrame({
        'mass_mean': bymanufacturer['m (kg)'].mean(),
        'total_new_registrations': bymanufacturer['r'].aggregate(np.sum),
        'emissions_mean': bymanufacturer['e (g/km)'].mean()
    }).reset_index()

df = df[df['total_new_registrations'] > 100000]

In [71]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.plotting import ColumnDataSource
from bokeh.models import HoverTool, NumeralTickFormatter

source = ColumnDataSource(df)

p = figure(
        x_axis_label='Specific CO2 Emissions Mean',
        y_axis_label='Total New Registrations'
    )

p.circle('emissions_mean', 'total_new_registrations', source=source, size=6)

p.yaxis.formatter=NumeralTickFormatter(format="0.0a")

hover = HoverTool(
            tooltips=[
                ('Manufacturer', '@Mh'),
                ('CO2 Emissions', '@emissions_mean'),
                ('Total New Registrations', '@total_new_registrations')
            ]
        )

p.add_tools(hover)

output_notebook()

show(p)

Loading BokehJS ...

In [72]:
source = ColumnDataSource(df)

p2 = figure(
        x_axis_label='Specific CO2 Emissions Mean',
        y_axis_label='Mass'
    )

p2.circle('emissions_mean', 'mass_mean', source=source, size=6)

# p.yaxis.formatter=NumeralTickFormatter(format="0.0a")

hover = HoverTool(
            tooltips=[
                ('Manufacturer', '@Mh'),
                ('CO2 Emissions', '@emissions_mean'),
                ('Mass', '@mass_mean')
            ]
        )

p2.add_tools(hover)

output_notebook()

show(p2)

Loading BokehJS ...

In [73]:
print(data['Ft'].unique())
data['Ft'] = data['Ft'].fillna('')
data['Ft'] = data['Ft'].apply(lambda x: x.lower())

petrol_data = data[data['Ft'].str.contains('petrol')]
diesel_data = data[data['Ft'].str.contains('diesel')]
electric_data = data[data['Ft'].str.contains('electric')]
lpg_data = data[data['Ft'].str.contains('lpg')]

['DIESEL' 'PETROL' 'ELECTRIC' 'NG-BIOMETHANE' 'Petrol-electric' 'Petrol'
 'Diesel' 'diesel' 'petrol' 'NG-biomethane' 'Electric' 'LPG' nan
 'petrol-electric' 'HYBRID/PETROL/E' 'PETROL-ELECTRIC' 'Petrol-Electric'
 'E85' 'Hydrogen' 'diesel-electric' 'electric' 'Diesel-electric'
 'Biodiesel' 'PETROL PHEV' 'PETROL/ELECTRIC' 'DIESEL-ELECTRIC'
 'NG-Biomethane' 'Diesel-Electric' 'OTHER']


In [74]:
petrol_data_bymanufacturer = petrol_data.groupby('Mh')

petrol_data_df = pd.DataFrame({
        'total_new_registrations': petrol_data_bymanufacturer['r'].aggregate(np.sum),
        'emissions_mean': petrol_data_bymanufacturer['e (g/km)'].mean()
    }).reset_index()

petrol_data_df = petrol_data_df[petrol_data_df['total_new_registrations'] > 100000]


electric_data_bymanufacturer = electric_data.groupby('Mh')

electric_data_df = pd.DataFrame({
        'total_new_registrations': electric_data_bymanufacturer['r'].aggregate(np.sum),
        'emissions_mean': electric_data_bymanufacturer['e (g/km)'].mean()
    }).reset_index()


diesel_data_bymanufacturer = electric_data.groupby('Mh')

diesel_data_df = pd.DataFrame({
        'total_new_registrations': diesel_data_bymanufacturer['r'].aggregate(np.sum),
        'emissions_mean': diesel_data_bymanufacturer['e (g/km)'].mean()
    }).reset_index()

In [75]:
source = ColumnDataSource(petrol_data_df)

p3 = figure(
        title='Petrol Data',
        x_axis_label='Specific CO2 Emissions Mean',
        y_axis_label='Total New Registrations',
        tools='box_select,lasso_select'
    )

p3.circle('emissions_mean', 'total_new_registrations', source=source, size=6)

p3.yaxis.formatter=NumeralTickFormatter(format="0.0a")

hover = HoverTool(
            tooltips=[
                ('Manufacturer', '@Mh'),
                ('CO2 Emissions', '@emissions_mean'),
                ('Total New Registrations', '@total_new_registrations')
            ]
        )

In [76]:
electric_data_bymanufacturer = electric_data.groupby('Mh')

electric_data_df = pd.DataFrame({
        'total_new_registrations': electric_data_bymanufacturer['r'].aggregate(np.sum),
        'emissions_mean': electric_data_bymanufacturer['e (g/km)'].mean()
    }).reset_index()


source = ColumnDataSource(electric_data_df)

p4 = figure(
        title='Electric Data',
        x_axis_label='Specific CO2 Emissions Mean',
        y_axis_label='Total New Registrations',
        tools='box_select,lasso_select'
    )

p4.circle('emissions_mean', 'total_new_registrations', source=source, size=6)

p4.yaxis.formatter=NumeralTickFormatter(format="0.0a")

hover = HoverTool(
            tooltips=[
                ('Manufacturer', '@Mh'),
                ('CO2 Emissions', '@emissions_mean'),
                ('Total New Registrations', '@total_new_registrations')
            ]
        )

In [77]:
diesel_data_bymanufacturer = electric_data.groupby('Mh')

diesel_data_df = pd.DataFrame({
        'total_new_registrations': diesel_data_bymanufacturer['r'].aggregate(np.sum),
        'emissions_mean': diesel_data_bymanufacturer['e (g/km)'].mean()
    }).reset_index()


source = ColumnDataSource(diesel_data_df)

p5 = figure(
        title='Diesel Data',
        x_axis_label='Specific CO2 Emissions Mean',
        y_axis_label='Total New Registrations',
        tools='box_select,lasso_select'
    )

p5.circle('emissions_mean', 'total_new_registrations', source=source, size=6)

p5.yaxis.formatter=NumeralTickFormatter(format="0.0a")

hover = HoverTool(
            tooltips=[
                ('Manufacturer', '@Mh'),
                ('CO2 Emissions', '@emissions_mean'),
                ('Total New Registrations', '@total_new_registrations')
            ]
        )

In [80]:
lpg_data_bymanufacturer = lpg_data.groupby('Mh')

lpg_data_df = pd.DataFrame({
        'total_new_registrations': lpg_data_bymanufacturer['r'].aggregate(np.sum),
        'emissions_mean': lpg_data_bymanufacturer['e (g/km)'].mean()
    }).reset_index()


source = ColumnDataSource(lpg_data_df)

p6 = figure(
        title='Liquefied Petroleum Gas Data',
        x_axis_label='Specific CO2 Emissions Mean',
        y_axis_label='Total New Registrations',
        tools='box_select,lasso_select'
    )

p6.circle('emissions_mean', 'total_new_registrations', source=source, size=6)

p6.yaxis.formatter=NumeralTickFormatter(format="0.0a")

hover = HoverTool(
            tooltips=[
                ('Manufacturer', '@Mh'),
                ('CO2 Emissions', '@emissions_mean'),
                ('Total New Registrations', '@total_new_registrations')
            ]
        )

In [79]:
from bokeh.layouts import gridplot

row1 =[p3, p4]

row2 = [p5, p6]

layout = gridplot([row1, row2], sizing_mode='scale_width')

# Link the x_range of p2 to p1: p2.x_range
p4.x_range = p3.x_range

# Link the y_range of p2 to p1: p2.y_range
p4.y_range = p3.y_range

# Link the x_range of p3 to p1: p3.x_range
p5.x_range = p3.x_range

p5.y_range = p3.y_range
# Link the y_range of p4 to p1: p4.y_range
p6.y_range = p3.y_range

p6.x_range = p3.x_range


output_notebook()

show(layout)

Loading BokehJS ...